In [1]:
#
##
### LOAD DATA

import pandas as pd
import numpy as np

def _optimize_numeric_dtypes(df):
    import pandas as pd
    float_cols = df.select_dtypes("float").columns
    int_cols = df.select_dtypes("integer").columns
    df[float_cols] = df[float_cols].\
        apply(pd.to_numeric, downcast="float")
    df[int_cols] = df[int_cols].\
        apply(pd.to_numeric, downcast="integer")
    return df

data = _optimize_numeric_dtypes(
    pd.read_parquet("../data/customer_model/retailrocket/"))
#data["target_cap"] = data["target_cap"].clip(-1000,10000)

#
##
### CONSTRUCT PROFIT TARGET
# NOTE: encapsulate this
config = {
    "gamma":{"alpha":2.04, "beta":202.04},
    "delta":7500, 
    "psi":{"alpha":6.12, "beta":3.15},
    "n_iter":1000,
    "seed":1}
    
gamma = config["gamma"]
delta = config["delta"]
psi = config["psi"]
n_iter = config["n_iter"]
seed = config["seed"] 

n_users = data.user_id.nunique()
sp = []
for i in range(n_iter):
    gamma_psi = pd.DataFrame.from_dict({
        "user_id":data.user_id.unique(),
        "gamma":np.random.beta(gamma["alpha"], gamma["beta"], size=n_users),
        "psi":np.random.beta(psi["alpha"], psi["beta"], size=n_users)})
    temp = data.merge(gamma_psi, on=["user_id"])
    temp["acp"] = (temp["target_event"]*temp["gamma"]*(temp["target_cap"]-delta)
        + (1-temp["target_event"])*(-temp["psi"]*delta))
    sp.append(temp.loc[:,["user_id", "week_step", "acp"]])
sp = pd.concat(sp)

#
##
### PUT IT TOGETHER

out_cols = ["user_id", "row_id", "target_event",
    "target_revenue", "week_step", "target_cap", "cap"]
feat_cols = [c for c in data.columns if c not in set(out_cols)]
target_cols = ["target_event","acp"]
data = data.merge(sp.groupby(["user_id","week_step"], as_index=False).acp.mean(), on=["user_id","week_step"])


trf = data.week_step>5
tef = data.week_step==5
X_train, y_train = data.loc[trf,feat_cols].values, data.loc[trf,target_cols].values
X_test, y_test = data.loc[tef,feat_cols].values, data.loc[tef,target_cols].values

In [2]:
#
##
### COMBINED NEURAL NETWORK

import numpy as np
import pandas as pd
from typing import List
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, PowerTransformer

class MultiOutputTransformer(BaseEstimator, TransformerMixin):

    def fit(self, y):
        if isinstance(y, pd.DataFrame):
            y = y.values
        y_class, y_reg = y[:, 0].reshape(-1,1), y[:, 1].reshape(-1,1)

        self.class_encoder_ = OneHotEncoder(sparse=False)
        self.reg_transformer_ = PowerTransformer()
        # Fit them to the input data
        self.class_encoder_.fit(y_class)
        self.reg_transformer_.fit(y_reg)
        # Save the number of classes
        self.n_classes_ = len(self.class_encoder_.categories_)
        self.n_outputs_expected_ = 2
        return self

    def transform(self, y):
        if isinstance(y, pd.DataFrame):
            y = y.values
        y_class, y_reg = y[:, 0].reshape(-1,1), y[:, 1].reshape(-1,1)
        # Apply transformers to input array
        y_class = self.class_encoder_.transform(y_class)
        y_reg = self.reg_transformer_.transform(y_reg)
        # Split the data into a list
        return [y_class, y_reg]

    def inverse_transform(self, y, return_proba=False):
        y_pred_reg = y[1]
        if return_proba:
            return y[0]
        else:
            y_pred_class = np.zeros_like(y[0])
            y_pred_class[np.arange(len(y[0])), np.argmax(y[0], axis=1)] = 1
            y_pred_class = self.class_encoder_.inverse_transform(y_pred_class)
        y_pred_reg = self.reg_transformer_.inverse_transform(y_pred_reg)
        return np.column_stack([y_pred_class, y_pred_reg])

    def get_metadata(self):
        return {
            "n_classes_": self.n_classes_,
            "n_outputs_expected_": self.n_outputs_expected_,
        }

from scikeras.wrappers import BaseWrapper
from tensorflow.keras.initializers import HeNormal, LecunNormal, HeNormal
from tensorflow.keras.layers import Input, Dense, BatchNormalization, concatenate, LeakyReLU
from tensorflow.keras import Model

class CombiNet(BaseWrapper):

    def __init__(self, activation = "selu",
        se_layers=1, se_units=256,
        re_layers=5, re_units=100,
        ce_layers=5, ce_units=100, cc_units=75,
        epochs=10, verbose=0,
        optimizer="adam", optimizer__clipvalue=1.0, **kwargs):
            super().__init__(**kwargs)
            self.activation = activation
            self.se_layers = se_layers
            self.se_units = se_units
            self.re_layers = re_layers
            self.re_units = re_units
            self.ce_layers = ce_layers
            self.ce_units = ce_units
            self.cc_units = cc_units
            self.epochs = epochs
            self.verbose = verbose
            self.prediction_scope_ = {"classification":0,"regression":1,"full":range(2)}

    def _get_weight_init(self):
        if isinstance(self.activation, LeakyReLU):
            
            init = HeNormal()
        elif self.activation in ["selu", "elu"]:
            init = LecunNormal()
        else:
            init = HeNormal()  
        return init

    def _keras_build_fn(self, compile_kwargs):
        weight_init = self._get_weight_init()

        # shared extraction
        inp = Input(shape=(self.n_features_in_))
        fe = inp
        for i in range(self.se_layers):
            fe = Dense(self.se_units, self.activation,
                kernel_initializer=weight_init)(fe)
            fe = BatchNormalization()(fe)
        # regression branch
        re = fe
        for i in range(self.re_layers):
            re = Dense(self.re_units, self.activation,
                kernel_initializer=weight_init)(re)
            re = BatchNormalization()(re)
        rr_head = Dense(1,"linear")(re)
        # classification branch
        ce = fe
        for i in range(self.ce_layers):
            ce = Dense(self.ce_units, self.activation,
                kernel_initializer=weight_init)(ce)
            ce = BatchNormalization()(ce)
        cc = Dense(self.cc_units, self.activation,
            kernel_initializer=weight_init)(concatenate([ce, re]))
        cc = BatchNormalization()(cc)
        cc_head = Dense(2, "softmax")(cc)

        model = Model(inputs=inp, outputs=[cc_head, rr_head])
        model.compile(loss=["categorical_crossentropy","mse"], loss_weights=[.5,.5],
            optimizer=compile_kwargs["optimizer"])
        return model
        
    @property
    def target_encoder(self):
        return MultiOutputTransformer()
        
    def predict_proba(self, X):
        X = self.feature_encoder_.transform(X)
        y_pred = self.model_.predict(X)
        return self.target_encoder_.inverse_transform(y_pred, return_proba=True)

    def predict(self, X, scope="classification"):
        X = self.feature_encoder_.transform(X)
        y_pred = self.model_.predict(X)
        y_pred = self.target_encoder_.inverse_transform(y_pred)
        return y_pred[:,self.prediction_scope_[scope]]

2022-07-14 18:11:26.334818: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-14 18:11:26.334868: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [14]:
cn = CombiNet(se_layers=2, se_units=512, batch_size=16,
    re_layers=1, ce_layers=1, optimizer="adam", epochs=200, verbose=1)

cn.fit(X_train,y_train)

Epoch 1/200
67/67 [==============================] - 2s 6ms/step - loss: 1.2741 - dense_63_loss: 0.6921 - dense_60_loss: 1.8561
Epoch 2/200
67/67 [==============================] - 0s 6ms/step - loss: 0.7415 - dense_63_loss: 0.5745 - dense_60_loss: 0.9084
Epoch 3/200
67/67 [==============================] - 0s 6ms/step - loss: 0.6739 - dense_63_loss: 0.5504 - dense_60_loss: 0.7973
Epoch 4/200
67/67 [==============================] - 0s 6ms/step - loss: 0.6497 - dense_63_loss: 0.5271 - dense_60_loss: 0.7724
Epoch 5/200
67/67 [==============================] - 0s 6ms/step - loss: 0.6389 - dense_63_loss: 0.5254 - dense_60_loss: 0.7524
Epoch 6/200
67/67 [==============================] - 0s 5ms/step - loss: 0.6067 - dense_63_loss: 0.5045 - dense_60_loss: 0.7089
Epoch 7/200
67/67 [==============================] - 0s 6ms/step - loss: 0.6290 - dense_63_loss: 0.5026 - dense_60_loss: 0.7554
Epoch 8/200
67/67 [==============================] - 0s 6ms/step - loss: 0.6109 - dense_63_loss: 0.5070 

CombiNet(
	model=None
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=16
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=200
	activation=selu
	se_layers=2
	se_units=512
	re_layers=1
	re_units=100
	ce_layers=1
	ce_units=100
	cc_units=75
)

In [20]:
from sklearn.metrics import f1_score
y_pred  = cn.predict(X_train, scope="classification")
print("r2:{:.2f}".format(f1_score(y_train[:,0], y_pred)))

34/34 [==============================] - 0s 2ms/step
r2:0.89
